In [11]:

import queue
import threading
import time

import pandas
from lxml import html
from selenium.webdriver import Chrome, Edge
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def GetChromeBrowser(isHeadless=True):
    opt = Options()
    if isHeadless:
        opt.add_argument("--headless")
    opt.add_argument("--mute-audio")
    opt.add_argument("--disable-notifications")
    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
    try:
        return Chrome(service=Service(ChromeDriverManager().install()), options=opt)
    except Exception as err:
        print(err)


def GetEdgeBrowser(isHeadless=True):
    options = EdgeOptions()
    if isHeadless:
        options.add_argument("--headless")
    options.add_argument("--mute-audio")
    options.add_argument("--disable-notifications")

    try:
        return Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as err:
        print(err)


resultQ = queue.Queue()
errorQ = queue.Queue()


def Scrapper(driver, url):
    driver.get(url)

    titleXpath = '//td[@class="navbar header-forum-description"]/descendant::strong/text()'
    lastButton = "//a[contains(text(),'Last')]/@href"

    try:
        lastButton_element = WebDriverWait(driver, 25).until(
            EC.visibility_of_all_elements_located((By.XPATH, lastButton)))
    except:
        pass

    d_data = {}
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * .2);")

    try:
        ctitleXpath_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, titleXpath)))
    except:
        pass


    page_source = driver.page_source
    doc = html.fromstring(page_source)

    d_data['title'] = '\n'.join(doc.xpath(titleXpath))
    d_data['price'] = '\n'.join(doc.xpath(titleXpath))
    d_data['url'] = url
    resultQ.put(d_data)


jobs = queue.Queue()


def WorkerLoop(progress_bar: tqdm):
    # driver = GetChromeBrowser(False)
    driver = GetEdgeBrowser(False)
    while not jobs.empty():
        url = jobs.get()
        try:
            Scrapper(driver, url)
        except:
            errorQ.put({'url': url})

            pass
        time.sleep(1)
        progress_bar.update()



data = pandas.read_excel('sample1..xlsx')

product_column = 'url'
urls = list(data[product_column])

[jobs.put(_) for _ in urls]
threads = []
pbar = tqdm(total=len(urls))

for worker in range(5):
    thread = threading.Thread(target=WorkerLoop, args=(pbar,))
    thread.start()
    threads.append(thread)

for t in threads:
    t.join()


def qToDf(q):
    l = []
    while not q.empty():
        l.append(q.get())
    [q.put(_) for _ in l]
    df = pandas.DataFrame(l)
    return df


df = qToDf(resultQ)
df.to_csv("sample scrapped.csv",index=False)



100%|██████████| 17/17 [01:00<00:00,  4.33s/it]

In [3]:
import pandas
data = pandas.read_excel('sample1..xlsx')
data

,url,Title,Last page
0,https://www.forum.hr/showthread.php?t=514447&p...,Kreme za ruke - Forum.hr,41.0
1,https://www.forum.hr/showthread.php?t=101410,Srce - pitanja razna - Forum.hr,59.0
2,https://www.forum.hr/showthread.php?p=52585193,Problemi s očima/vidom - Forum.hr,66.0
3,https://www.forum.hr/showthread.php?t=1250119&...,NaN,NaN
4,https://www.forum.hr/showthread.php?t=712949,NaN,NaN
5,https://www.forum.hr/showthread.php?t=832459,NaN,NaN
6,https://www.forum.hr/showthread.php?t=126102,NaN,NaN
7,https://www.forum.hr/showthread.php?t=77314,NaN,NaN
8,https://www.forum.hr/showthread.php?t=162548&p...,NaN,NaN
9,https://www.forum.hr/showthread.php?t=18539,NaN,NaN
